# Test Code: Ignore

In [54]:
# import numpy as np
# from PIL import Image

# test_pic = 'images/chaparral/chaparral56.tif'

# img = Image.open(test_pic)
# img_arr = np.array(img) 
# res_len = len(img_arr)
# res_width = len(img_arr[0])

# R = np.empty([res_len, res_width], dtype=int)
# G = np.empty([res_len, res_width], dtype=int)
# B = np.empty([res_len, res_width], dtype=int)


# for i in range(res_len):
#     for j in range(res_width):
#         R[i,j] = img_arr[i,j,0]
#         G[i,j] = img_arr[i,j,1]
#         B[i,j] = img_arr[i,j,2]

# colours = np.array([R, G, B])
# colours


array([[[174, 158, 127, ..., 175, 174, 173],
        [177, 152, 104, ..., 176, 175, 172],
        [170, 121,  89, ..., 174, 177, 176],
        ...,
        [187, 191, 188, ...,  97, 110, 128],
        [185, 187, 189, ...,  82, 125, 143],
        [174, 183, 195, ...,  84, 117, 134]],

       [[150, 138, 111, ..., 147, 148, 148],
        [152, 129,  86, ..., 147, 148, 147],
        [148, 100,  70, ..., 148, 151, 151],
        ...,
        [167, 166, 160, ...,  81,  93, 109],
        [166, 164, 160, ...,  67, 108, 125],
        [153, 159, 167, ...,  71, 100, 119]],

       [[127, 115,  88, ..., 123, 122, 127],
        [130, 105,  62, ..., 122, 120, 123],
        [120,  77,  46, ..., 125, 125, 125],
        ...,
        [145, 144, 138, ...,  60,  77,  91],
        [143, 141, 139, ...,  47,  93, 109],
        [132, 138, 147, ...,  55,  85, 104]]])

In [4]:
# For now primitively split into buckets
colour_vals = 256
levels = 5
bins = np.linspace(0, 256, levels + 1)
colours_bucketed = np.digitize(colours, bins)
colours_bucketed -= np.ones([3, res_len, res_width], dtype=int)
colours_bucketed
# test
colours_bucketed = np.zeros([3,8, 8], dtype=int)
colours_bucketed[0,1] = [1,1,1,1,0,0,0,0]
colours_bucketed

NameError: name 'np' is not defined

In [5]:
# Can do it randomly for different levels
import math

def find_fd_old(level, colour, res_len, res_width):
    iterations = int(math.log2(res_len)) + 1
    scale_sets = [set() for i in range(iterations)] # test this out linearly as well?
    for i in range(res_len):
        for j in range(res_width):
            if colours_bucketed[colour, i,j] == level:
                for scale in range(iterations):
                    scale_size = 2**scale
                    tuple_i = i//scale_size
                    tuple_j = j//scale_size
                    scale_sets[scale].add((tuple_i, tuple_j))
    
   
    scale_nums = [len(i) for i in scale_sets]
    print(scale_nums)
    if (1 not in scale_nums): return 0
    else:
        scale_nums = scale_nums[:scale_nums.index(1)+1]
        if (len(scale_nums) == 1): return 0
        else:
            log_y = [math.log2(i) for i in scale_nums]
            log_x = [i for i in range(len(log_y))]
            m,b = np.polyfit(log_x, log_y, 1)
            return -m

# print(find_fd(1,0, 8, 8))
    

In [6]:
# For each level
# - for each pixel, dump the right value in the sets available
# - find total for each set
# - find log log
# - find the slope of the line as the dimension.

import math
res_len = 8
res_width = 8

FD = np.empty([levels, 3])

for level in range(levels):
    for colour in range(3):
        FD[level, colour] = find_fd(level, colour, res_len, res_width)
        
print(FD)

NameError: name 'np' is not defined

# MAIN CODE

## Calculates vector fractal dimension for a given colour

In [7]:
def find_fd(colours_bucketed, levels, colour, res_len, res_width):
    fd = []
    iterations = int(math.log2(res_len)) + 1
    scale_sets = [[set() for i in range(iterations)] for j in range(levels)]
    for i in range(res_len):
        for j in range(res_width):
            for scale in range(iterations):
                level = colours_bucketed[i, j, colour] 
                scale_size = 2**scale
                tuple_i = i//scale_size
                tuple_j = j//scale_size
                scale_sets[level][scale].add((tuple_i, tuple_j))
    
    scale_nums = [list(map(len, scale_sets[i])) for i in range(levels)]
    colour_vals = [scale_nums[i][0] for i in range(levels)]

    for level in range(levels):
        if (1 not in scale_nums[level]): fd.append(0)
        else:
            scale_nums[level] = scale_nums[level][:scale_nums[level].index(1)+1]
            if (len(scale_nums[level]) == 1): fd.append(0)
            else:
                log_y = [math.log2(i) for i in scale_nums[level]]
                log_x = [i for i in range(len(log_y))]
                m,b = np.polyfit(log_x, log_y, 1)
                fd.append(-m)
    return (fd, colour_vals)
    
    
# print(find_fd(colours_bucketed, 5, 0, 256, 256))      

## Returns all fractal dimensions, and colour array for a given image

In [8]:
def find_fd_array_colours(colours_bucketed, levels, res_len, res_width):
    FD = [0,0,0]
    colour_values = [0,0,0]
    for colour in range(3):
        FD[colour],colour_values[colour]  = find_fd(colours_bucketed, levels, colour, res_len, res_width)
            
    return (FD, colour_values)

# print(find_fd_array_colours(colours_bucketed, 5, 256, 256))

## Buckets the colours into levels

In [9]:
def bucket_colours(image_array, res_len, res_width, levels, colour_vals):
    bins = np.linspace(0, colour_vals, levels+1)
    colours_bucketed = np.digitize(image_array, bins)
    colours_bucketed -= np.ones([res_len, res_width, 3], dtype = int)
    return colours_bucketed
    
# colours_bucketed = bucket_colours(img_arr, 256, 256, 5, 256)
# print(colours_bucketed)

# Image Extraction: Code starts here
## Define all the variables at the start of this cell to use for your own dataset

In [6]:
import os
import numpy as np
import itertools as it
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import time

# VARIABLES TO DEFINE

# Resolution should be the same for all images
# Currently only works for equal length and width
# Note: Images not of (the shape given below + RGB format) are ignored
res_len = 256
res_width = 256

# Pick levels you want to split intensity into
levels = 5
colour_levels = 256

# Folder path where images are stored
folder_path = 'images/'

class_names = os.listdir(folder_path)

for levels in range(3, 11):
    start = time.process_time()
    X_all = []
    y_all = []
    for categ in range(len(class_names)):

        for image_path in os.listdir(folder_path+class_names[categ]):
#             print(image_path)
            image_add = folder_path+class_names[categ]+"/"+image_path
            image = Image.open(image_add)
            image_array = np.array(image)

            if (image_array.shape != (res_len, res_width, 3)):
#                 print(image_array.shape)
                continue

            colours_bucketed = bucket_colours(image_array, res_len, res_width, levels, colour_levels) 
            fd_array, colour_values = find_fd_array_colours(colours_bucketed, levels, res_len, res_width) 
            fd_array = list(it.chain(*fd_array))
            colour_values = list(it.chain(*colour_values))
            final_feature_vec = fd_array + colour_values

            X_all.append(final_feature_vec)
            y_all.append(categ)
            
    X_all = [k[:15] for k in X_all]
    X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = 0.25, random_state=42)
    
    clf = MLPClassifier()
    clf.fit(X_train, y_train) 
    y_pred = clf.predict(X_test)
    ac = accuracy_score(y_test, y_pred)
    t = time.process_time() - start
    print("levels:" + str(levels) + "      " + str(ac)+"      "+"time taken:    "+str(t))
        
    

levels:3      0.26459143968871596      time taken:    3709.1279423359997
levels:4      0.17898832684824903      time taken:    3742.7611898399996


/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


levels:5      0.39105058365758755      time taken:    3725.607362146001


/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


levels:6      0.4046692607003891      time taken:    3948.424121935999


...

/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


levels:8      0.4027237354085603      time taken:    2853.256910442


/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


levels:9      0.4143968871595331      time taken:    2665.5528566409994
levels:10      0.4357976653696498      time taken:    2680.599535813999


/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [12]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import time
import os
import numpy as np
import itertools as it
from PIL import Image

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(100),
    SVC(kernel="linear", C=0.025, max_iter = 1000),
    SVC(gamma=2, C=1, max_iter = 1000),
    GaussianProcessClassifier(max_iter_predict = 1000),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    MLPClassifier(max_iter = 1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]


res_len = 256
res_width = 256

# Pick levels you want to split intensity into
colour_levels = 256

# Folder path where images are stored
folder_path = 'images/'

class_names = os.listdir(folder_path)


X_all = []
y_all = []
levels = 10
for categ in range(len(class_names)):
    for image_path in os.listdir(folder_path+class_names[categ]):
        image_add = folder_path+class_names[categ]+"/"+image_path
        image = Image.open(image_add)
        image_array = np.array(image)

        if (image_array.shape != (res_len, res_width, 3)):
            continue

        colours_bucketed = bucket_colours(image_array, res_len, res_width, levels, colour_levels) 
        fd_array, colour_values = find_fd_array_colours(colours_bucketed, levels, res_len, res_width) 
        fd_array = list(it.chain(*fd_array))
        colour_values = list(it.chain(*colour_values))
        final_feature_vec = fd_array + colour_values
        X_all.append(final_feature_vec)
        y_all.append(categ)

NameError: name 'train_test_split' is not defined

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_all = [k[:15] for k in X_all]
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = 0.25, random_state=42)

In [15]:
for name, clf in zip(names, classifiers): 
    start = time.process_time()
    clf.fit(X_train, y_train) 
    y_pred = clf.predict(X_test)
    ac = accuracy_score(y_test, y_pred)
    t = time.process_time() - start
    print("classifier:" + str(name) + "      " + str(ac)+"      "+"time taken:    "+str(t))

classifier:Nearest Neighbors      0.35019455252918286      time taken:    0.15679512399992745
classifier:Linear SVM      0.22568093385214008      time taken:    0.18024569200042606
classifier:RBF SVM      0.5797665369649806      time taken:    0.30941047999976945
classifier:Gaussian Process      0.5136186770428015      time taken:    134.801403854
classifier:Decision Tree      0.46108949416342415      time taken:    0.2714209999999184
classifier:Random Forest      0.6478599221789884      time taken:    0.6033111020001343


/usr/local/lib/python3.5/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


classifier:Neural Net      0.5038910505836576      time taken:    35.52533771200024
classifier:AdaBoost      0.14980544747081712      time taken:    0.6342554710004151
classifier:Naive Bayes      0.3715953307392996      time taken:    0.017041748999872652
classifier:QDA      0.5428015564202334      time taken:    0.028277395000259276


In [16]:
print(X_train)

[[1.1999555783313036, 1.5192424047842876, 1.4762467201422425, 1.4895208087821477, 1.5791701029066696, 1.7553820374580238, 1.6346128115883996, 1.6073337817262636, 1.6422702350579754, 1.4832190535785124, 1.1633355587609033, 1.417298164092357, 1.5376266899559374, 1.4712285334881519, 1.5457629475684078], [0, 0, 0.20148586313816413, 1.0888720640339988, 1.3357313895406009, 1.7436688072935453, 1.9310224243016696, 1.328872047783787, 0, 0, 0, 0, 0.8194125503022407, 1.1879082257273959, 1.5219585204573987], [1.2717390115510365, 1.4450575678654045, 1.5144640317387699, 1.6547141841165254, 1.6991915232015833, 1.6800940980633912, 1.6821956335989052, 1.6321196046242665, 1.4327327308002036, 0.4584628848144216, 1.1629888735469587, 1.4294448211996464, 1.5527722094790886, 1.65533674728557, 1.686038726406199], [0, 0.23333333333333325, 1.3342873430827518, 1.6795471407803375, 1.7405836329729818, 1.7064541613549857, 1.7490593916602215, 1.7529699349357022, 1.2283729905256127, 1.3984080251670699, 0, 1.148925023

In [18]:
print(y_train)

[5, 5, 17, 15, 4, 2, 12, 2, 4, 16, 3, 6, 13, 4, 19, 5, 16, 14, 18, 10, 7, 5, 6, 6, 1, 1, 6, 6, 19, 18, 14, 19, 2, 17, 14, 9, 3, 2, 7, 6, 20, 5, 3, 7, 12, 11, 9, 0, 7, 10, 20, 17, 15, 9, 4, 8, 2, 12, 11, 15, 13, 19, 9, 14, 14, 16, 15, 9, 0, 2, 11, 1, 9, 7, 10, 1, 12, 5, 20, 15, 8, 16, 12, 13, 19, 16, 0, 15, 14, 14, 10, 4, 11, 5, 7, 8, 20, 7, 15, 9, 3, 17, 4, 7, 12, 3, 2, 19, 7, 16, 8, 13, 1, 19, 11, 13, 8, 10, 5, 5, 15, 13, 13, 14, 19, 13, 10, 2, 16, 14, 6, 6, 9, 19, 17, 18, 10, 18, 14, 18, 6, 12, 15, 6, 12, 17, 14, 10, 5, 13, 2, 19, 3, 11, 2, 5, 19, 9, 2, 14, 16, 0, 6, 16, 19, 16, 3, 5, 12, 11, 16, 13, 9, 13, 12, 1, 13, 8, 5, 5, 0, 7, 10, 3, 19, 17, 0, 17, 10, 3, 0, 7, 18, 18, 20, 4, 4, 15, 2, 19, 20, 16, 14, 19, 10, 17, 8, 9, 13, 12, 17, 3, 4, 8, 11, 13, 13, 1, 7, 12, 0, 20, 20, 4, 19, 0, 15, 6, 6, 2, 1, 20, 19, 10, 1, 9, 6, 16, 0, 4, 18, 2, 16, 4, 15, 8, 6, 0, 12, 7, 11, 2, 20, 13, 11, 3, 5, 15, 7, 8, 12, 3, 6, 1, 12, 3, 0, 0, 4, 1, 11, 19, 15, 14, 13, 17, 5, 18, 1, 11, 18, 18, 9, 12

In [19]:
print(X_test)

[[0, 0, 0.5471978860721497, 1.5077674205814942, 1.7489658716890433, 1.8278954890900156, 1.6929116019602748, 1.6367454803932366, 1.1081624845495552, 0, 0, 0, 0.3911545484602917, 1.4496378153956482, 1.648428115382642], [0, 1.4684232692204562, 1.330459069060353, 1.5391401145844068, 1.5811903997708403, 1.7345440837330572, 1.7998567950130064, 1.6571408279080093, 1.225510353865685, 0.8011739696869147, 0, 1.4723340311562252, 1.3727562627186205, 1.5563531080905215, 1.65555043292177], [0, 0, 0, 1.5729472471514867, 1.751122487313872, 1.8583109572230436, 1.1656507267565293, 1.3192180143597283, 1.2515996250496133, 1.5048526912611064, 0, 0, 1.404559153788693, 1.6898567409574192, 1.9028898967240286], [1.7623397459564496, 1.8784712359727407, 1.8447546760672227, 1.5183707276723062, 0.22157447408779715, 0, 0, 0, 0, 0, 1.5551917961779393, 1.8383478571538294, 1.8411326201203106, 1.7773132314849862, 1.4444181587093146], [1.0349854042357625, 1.3730760904463024, 1.749184786395114, 1.6904804280809507, 1.6511

In [20]:
print(y_test)

[1, 17, 2, 11, 15, 1, 14, 1, 6, 2, 11, 18, 6, 13, 5, 7, 5, 16, 0, 5, 4, 16, 14, 17, 17, 11, 19, 9, 9, 3, 12, 3, 10, 19, 8, 14, 17, 9, 14, 4, 11, 19, 18, 5, 12, 2, 17, 19, 9, 18, 8, 10, 5, 6, 0, 8, 0, 11, 1, 7, 16, 20, 2, 9, 2, 3, 17, 13, 20, 4, 10, 9, 5, 16, 6, 0, 16, 6, 7, 10, 13, 4, 8, 7, 5, 16, 16, 3, 6, 10, 7, 8, 3, 20, 16, 3, 3, 13, 14, 20, 18, 19, 11, 8, 16, 15, 7, 7, 4, 16, 2, 0, 2, 5, 7, 8, 1, 2, 7, 20, 15, 1, 1, 8, 6, 7, 5, 3, 19, 16, 3, 14, 15, 20, 12, 10, 9, 4, 9, 2, 5, 7, 16, 20, 19, 14, 13, 10, 6, 17, 9, 2, 20, 5, 5, 18, 5, 10, 13, 9, 14, 9, 16, 7, 17, 16, 9, 2, 13, 15, 10, 17, 15, 11, 1, 12, 19, 11, 16, 17, 9, 16, 19, 5, 2, 20, 15, 12, 20, 13, 3, 1, 16, 11, 6, 4, 11, 3, 1, 3, 5, 10, 3, 6, 14, 10, 16, 1, 20, 20, 18, 2, 13, 20, 18, 0, 13, 5, 11, 14, 8, 15, 6, 10, 3, 11, 4, 9, 7, 11, 3, 2, 4, 17, 4, 14, 20, 13, 15, 1, 10, 0, 3, 6, 4, 16, 18, 8, 20, 0, 1, 8, 2, 18, 18, 11, 4, 12, 12, 5, 17, 3, 20, 7, 9, 4, 2, 6, 16, 10, 20, 7, 10, 17, 11, 4, 7, 3, 12, 17, 18, 2, 3, 4, 8, 3, 3

## ML on the extracted features!

In [110]:
len(X_all)

2056

In [111]:
len(Y_all)

NameError: name 'Y_all' is not defined

In [132]:
X_all = [k[:15] for k in X_all]

In [133]:
from sklearn.model_selection import train_test_split


In [134]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = 0.25, random_state=42)

In [135]:
X_train

[[0,
  1.3265046475084443,
  1.9846854538289287,
  1.6270816279445894,
  0,
  0,
  1.3269950165825581,
  1.9838001768687887,
  1.6377901813997444,
  0,
  0,
  1.123782539885896,
  1.988520375830933,
  1.6254147699779153,
  0],
 [1.9749943067633102,
  1.7217251443278516,
  0,
  0,
  0,
  1.9358712336679509,
  1.8332250838699673,
  0.578903016355603,
  0,
  0,
  1.934466508237083,
  1.8450119649323455,
  0.3575406053110733,
  0,
  0],
 [0,
  1.6763355240606692,
  1.7435211018611074,
  1.8057111896209779,
  1.7537898642836234,
  1.3289404026720137,
  1.6922815065359473,
  1.7705099223077463,
  1.8658501980875968,
  1.4000228914703634,
  1.6034684949430054,
  1.6524272047645823,
  1.7981609269018493,
  1.8355555396836438,
  0.911518148524459],
 [1.5480000290323856,
  1.926048877423725,
  1.696909605586235,
  1.5541749241185363,
  1.4636520285763068,
  1.4947031456060285,
  1.9248549469759508,
  1.7196136328368312,
  1.5561795432256778,
  1.4747958233851142,
  1.3767919019413701,
  1.925377

In [139]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier()

In [140]:
clf.fit(X_train, y_train)

/home/tinkidinki/.local/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [141]:
y_pred = clf.predict(X_test)

In [145]:
y_pred


array([ 1,  4, 15,  5, 15,  7, 20, 15,  1,  2, 12,  0,  8,  6, 14,  7, 14,
       15, 18, 14,  2,  7, 14, 17, 18,  6,  9, 16, 18,  5, 12,  0, 12,  9,
       19, 10, 17,  1,  2, 18,  5, 10, 19,  5, 14, 17,  2, 10, 14,  0, 18,
       10, 11,  6,  4,  4, 13, 11,  2,  7,  9, 20,  9, 10, 17,  6, 18, 13,
       20,  9, 16, 16,  5, 12,  0, 10, 16,  6,  7, 17, 13, 13,  1,  7,  5,
        2,  7,  5,  6, 15,  7,  2, 20, 20, 16, 15, 20, 13, 14, 20,  2, 17,
       14, 13, 10,  1,  7,  7, 18, 18, 17,  0, 17, 14,  7,  3, 10, 17,  7,
       20, 16,  1,  1,  3,  6,  7,  5, 10, 16,  8,  4, 14, 15, 20, 15, 10,
       17, 18, 15, 17,  5,  7, 17, 20,  9, 14, 17, 12,  5, 18, 12,  7, 20,
        3,  5,  8,  5, 15, 15, 16,  3, 12,  7,  7, 17,  7,  1,  2,  3,  7,
       10,  2, 16, 11,  1, 12, 17, 14, 16, 17, 14,  7, 11,  5,  7, 20, 15,
       12,  1, 11, 14,  1, 16, 11,  6, 18,  6,  5,  0,  3, 20, 16, 19,  0,
        9, 10,  9,  1, 20, 20, 13, 17,  3, 20, 13,  0, 18, 14,  4,  7,  0,
       15,  6, 10,  0, 12

In [146]:
y_test

[1,
 17,
 2,
 11,
 15,
 1,
 14,
 1,
 6,
 2,
 12,
 18,
 6,
 14,
 5,
 7,
 5,
 16,
 0,
 5,
 4,
 16,
 14,
 17,
 17,
 11,
 19,
 9,
 9,
 3,
 12,
 3,
 10,
 19,
 8,
 14,
 17,
 9,
 15,
 4,
 11,
 19,
 18,
 5,
 12,
 2,
 17,
 19,
 9,
 18,
 8,
 10,
 5,
 6,
 0,
 8,
 0,
 11,
 1,
 7,
 16,
 20,
 2,
 10,
 2,
 3,
 17,
 13,
 20,
 4,
 10,
 9,
 5,
 16,
 6,
 0,
 16,
 6,
 7,
 10,
 13,
 4,
 8,
 7,
 5,
 16,
 16,
 3,
 6,
 10,
 7,
 8,
 3,
 20,
 16,
 3,
 3,
 13,
 14,
 20,
 18,
 19,
 11,
 9,
 16,
 15,
 7,
 7,
 4,
 17,
 2,
 0,
 2,
 5,
 7,
 8,
 1,
 2,
 7,
 20,
 16,
 1,
 1,
 8,
 6,
 7,
 5,
 3,
 19,
 17,
 3,
 14,
 15,
 20,
 13,
 10,
 9,
 4,
 9,
 2,
 5,
 7,
 17,
 20,
 19,
 14,
 13,
 10,
 5,
 17,
 10,
 2,
 20,
 5,
 5,
 18,
 5,
 10,
 13,
 9,
 14,
 9,
 16,
 7,
 17,
 16,
 9,
 2,
 13,
 15,
 10,
 17,
 15,
 11,
 1,
 12,
 19,
 11,
 16,
 17,
 9,
 16,
 19,
 5,
 2,
 20,
 15,
 12,
 20,
 13,
 3,
 1,
 16,
 11,
 6,
 4,
 11,
 3,
 1,
 3,
 5,
 10,
 3,
 6,
 15,
 10,
 16,
 1,
 20,
 20,
 18,
 2,
 13,
 20,
 18,
 0,
 13,
 5,
 11,
 14,
 8,
 15

In [147]:
from sklearn.metrics import accuracy_score

In [148]:
accuracy_score(y_test, y_pred)

0.39299610894941633